In [14]:
# Import Libraries
import os.path
import numpy as np
from tensorflow.python.platform import gfile
import pypcd
import os
import subprocess
import pandas as pd
from pandas import DataFrame


In [15]:
# Find number of PointClouds

point_dir = '/home/johan/Documents/Alignment/Points_csv/'
#se crea un vector vacio para guardar los nombres de las imágenes.
file_list = []
#se obtienen todos los nombre de las imagenes en la carpeta train que tengan extensión .csv
file_glob = os.path.join(point_dir, '*.csv')
#se organizan en el vector file_list todos las direcciones de las imágenes 
#encontrados dentro de la carpeta train
file_list.extend(gfile.Glob(file_glob))
print(len(file_list))
print(type(file_list[0]),file_list[0])

410
<class 'str'> /home/johan/Documents/Alignment/Points_csv/pointscsv2954.csv


In [16]:
# Alignt PCs
#para cada pointcloud dentro del vector file_list
a = 0 
for point_dir in file_list:
    if a == 0:
        refe = pd.read_csv(point_dir)
        df = DataFrame(refe, columns= ['x', 'y','z'])
        export_csv = df.to_csv ('/home/johan/Documents/Alignment/Points_csv/refe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
        #print (df)
        
    else:
        pointcloud1_path_refe = "/home/johan/Documents/Alignment/Points_csv/refe.csv"
        refe = pd.read_csv(pointcloud1_path_refe)
        refe = np.array(refe)
        print ("refe", refe.shape) 
        
        pointcloud2_path_in = point_dir
        icp_path="/home/johan/Libraries/libpointmatcher/build/examples"

        #--------------- correr el ejecutable de c++ ------------------
        #runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path,pointcloud2_path],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path_refe,pointcloud2_path_in,'OutPc'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        runicp.wait()
        #-------------leer los datos de devuelve en el terminal------
        #stdout, stderr = runicp.communicate()
        print('ICP finished succesfully')
        print('------------------------------------------------------')
        #------------------------------------------------------------
            
        refe2 = pd.read_csv('/home/johan/repos/GitHub/3D-Reconstruction/refe_Out.csv')
        refe2 = np.array(refe2)
        refe2 = refe2[1:refe2.shape[0],:]
        print('refe2',refe2.shape)

        refetotal = np.vstack((refe,refe2))
        print('refetotal',refetotal.shape)
        np.savetxt('/home/johan/Documents/Alignment/Points_csv/refe.csv', refetotal)

    
    a = a +1
    print('a:',a)
        
        
        
 

    
    

a: 1
refe (23740, 3)
ICP finished succesfully
------------------------------------------------------
refe2 (25320, 3)
refetotal (49060, 3)


TypeError: Mismatch between array dtype ('object') and format specifier ('%.18e %.18e %.18e')